In [1]:
import numpy as np
import cv2
import time

from tkinter import *
from PIL import Image, ImageTk

from matplotlib import pyplot as plt

In [5]:
def calculate_disparity_matrix(imgL, imgR,min_disp, max_disp, disp_size, window_radius):
    num_disp = max_disp-min_disp
    h, w = imgL.shape
    
    croph = [(h-disp_size)//2,(h+disp_size)//2]
    cropw = [(w-disp_size)//2,(w+disp_size)//2]

    kernel = np.ones([window_radius, window_radius])

    ref_img_cropped = imgL[croph[0]:croph[1],cropw[0]:cropw[1]]
    C = np.zeros(
        [disp_size, disp_size, num_disp])

    for d in range(0, num_disp):
        # shift image
        translation_matrix = np.float32([[1, 0, d], [0, 1, 0]])
        shifted_image = cv2.warpAffine(
            imgR, translation_matrix,
            (imgR.shape[1], imgR.shape[0]))
        shifted_image_cropped = shifted_image[croph[0]:croph[1],cropw[0]:cropw[1]]
        # calculate squared differences
        SAD = abs(np.float32(ref_img_cropped) - np.float32(shifted_image_cropped))
        # convolve with kernel and find SAD at each point
        C[:, :, d] = cv2.filter2D(SAD, -1, kernel)
        

    disparity = np.argmin(C, axis=2)
    return disparity

In [3]:
#Parameters

#Stereo cameras
focal_length = 567.2 #pixels
baseline = 92.226 #mm

#Stereo matching algorithm
min_disp=0
max_disp=128
num_disp = max_disp-min_disp
disparity_area_size = 150
window_radius = 8

In [6]:

#win = tk()
#win.geometry("700x550")

vidL = cv2.VideoCapture('../data/robotL.avi')
vidR = cv2.VideoCapture('../data/robotR.avi')
if (vidL.isOpened()==False or vidR.isOpened()==False):
  print("Error opening video")

# Read until video is completed
while(vidL.isOpened()):
      # Capture frame-by-frame
      retL, frameL = vidL.read()
      retR, frameR = vidR.read()
      if retL == True and retR == True:
        frameL_greyscale=cv2.cvtColor(frameL, cv2.COLOR_BGR2GRAY)
        frameR_greyscale=cv2.cvtColor(frameR, cv2.COLOR_BGR2GRAY)
        
    
        start = time.time()

        disparity = calculate_disparity_matrix(frameL_greyscale, frameR_greyscale,min_disp, max_disp, disparity_area_size, window_radius)
        main_disp = np.mean(disparity)
        
        #distance = get_distance_from_disparity(main_disp, focal_length, baseline)
        print("FPS: ", 1/(time.time()-start))
        #print("Distance to obstacle(m): ", distance)
		    
        # Scale disparity values for visualization 
        disparity_visual = np.uint8(disparity * 255 / num_disp)
          
        # Display the resulting frame
        #print(frameL.shape)
        #img = cv2.imshow('Image', frameL)
        cv2.imshow('Disparity', disparity_visual)

        #Rearrange colors
        """blue,green,red = cv2.split(frameL)
        img = cv2.merge((red,green,blue))
        im = Image.fromarray(img)
        imgtk = ImageTk.PhotoImage(image=im)

        #Create a Label to display the image
        Label(win, image= imgtk).pack()
        win.mainloop()"""
        # Press Q on keyboard to  exit
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break

      # Break the loop
      else: 
          break

# When everything done, release the video capture object
vidL.release()
vidR.release()
# Closes all the frames
cv2.destroyAllWindows()


FPS:  5.710622840291147
FPS:  5.959672113910435
FPS:  4.845002350707002
FPS:  6.771319068421851
FPS:  6.593925978562647
FPS:  6.798604717507164
FPS:  6.7349706550625035
FPS:  6.767189415940626
FPS:  4.6733556696263925
FPS:  6.859333348596997
FPS:  6.761636539208685
FPS:  6.079802862837471
FPS:  6.611971309214156
FPS:  6.072031015203588
FPS:  6.51464446194397
FPS:  6.867925640526011
FPS:  6.701440050105371
FPS:  6.86247574419906
FPS:  6.762454170522738
FPS:  5.9173131645737485
FPS:  6.692340585768718
FPS:  6.681413349432981
FPS:  6.710800501753577
FPS:  6.706368989848438
FPS:  6.2878122128226694
FPS:  6.745867771918058
FPS:  6.465836374091048
FPS:  5.574240740839185
FPS:  6.9114284066472775
FPS:  6.671073433747395
FPS:  6.837439194564353
FPS:  5.923898812340138
FPS:  6.851310867541123
FPS:  6.200445855729601
FPS:  6.929023134717936
FPS:  6.848514140160669
FPS:  6.905249848536733
FPS:  6.717399590643088
FPS:  6.77776395247183
FPS:  6.7488744645862635
FPS:  6.11126830952773
FPS:  6.864845